In [1]:
!nvidia-smi

Sat Oct 24 13:05:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls drive/'My Drive'/'WEB_Ask_06devbros'/'ai'/'chatbot'

checkpoint  Dockerfile	exp.ipynb   model	requirements.txt
data	    docs	main.ipynb  preprocess	service
dataloader  example	main.py     README.md	train


In [4]:
!pip install -r drive/'My Drive'/'WEB_Ask_06devbros'/'ai'/'chatbot'/requirements.txt

     |████████████████████████████████| 778kB 1.7MB/s 
     |████████████████████████████████| 2.9MB 3.2MB/s 
     |████████████████████████████████| 1.1MB 11.8MB/s 
     |████████████████████████████████| 890kB 15.6MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251562 sha256=412f1c5e31f106132be3a5a6d0a16355dafe252a45e2bdfd4d25d32241e6a7c3
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=cdb4f5ec40e95c6a0e16b724b3f73611ba062a60e91a5139e6b3bf5abbc59584
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built kss sacremoses
ERROR: transformers 3.0.2 has requirement tokenizers==0.8.1.rc1, but you'll have tokenizers 0.9.2 which is incompatible.


In [5]:
!pip install --upgrade tokenizers==0.8.1.rc1

     |████████████████████████████████| 3.0MB 1.4MB/s 
  Found existing installation: tokenizers 0.9.2
    Uninstalling tokenizers-0.9.2:
      Successfully uninstalled tokenizers-0.9.2


In [6]:
import sys
sys.path.append('/content/drive/My Drive/WEB_Ask_06devbros/ai/chatbot')

In [7]:
import torch
import torch.nn as nn
import random

import torch
from transformers import (
  ElectraConfig,
  ElectraTokenizer
)
from model.koelectra import koElectraForSequenceClassification,koelectra_input

In [8]:
def load_wellness_answer():
  root_path = '..'
  category_path = f"{root_path}/data/wellness_dialog_category.txt"
  answer_path = f"{root_path}/data/wellness_dialog_answer.txt"

  c_f = open(category_path,'r')
  a_f = open(answer_path,'r')

  category_lines = c_f.readlines()
  answer_lines = a_f.readlines()

  category = {}
  answer = {}
  for line_num, line_data in enumerate(category_lines):
    data = line_data.split('    ')
    category[data[1][:-1]]=data[0]

  for line_num, line_data in enumerate(answer_lines):
    data = line_data.split('    ')
    keys = answer.keys()
    if(data[0] in keys):
      answer[data[0]] += [data[1][:-1]]
    else:
      answer[data[0]] =[data[1][:-1]]

  return category, answer

In [24]:
root_path='/content/drive/My Drive/WEB_Ask_06devbros/ai/chatbot'
checkpoint_path =f"{root_path}/checkpoint"
save_ckpt_path = f"{checkpoint_path}/koelectra-wellness-text-classification-20201023.pth"
model_name_or_path = "monologg/koelectra-base-discriminator"

#답변과 카테고리 불러오기
category = []
idx = -1
with open(root_path+'/data/wellness_data_for_text_classification.txt', 'r') as f:
  while True:
    line = f.readline()
    if not line:
      break
    datas = line.strip().split("\t")
    if datas[1] != str(idx):
      category.append(datas[2])
      idx += 1

ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

# 저장한 Checkpoint 불러오기
checkpoint = torch.load(save_ckpt_path, map_location=device)

# Electra Tokenizer
tokenizer = ElectraTokenizer.from_pretrained(model_name_or_path)

electra_config = ElectraConfig.from_pretrained(model_name_or_path)
model = koElectraForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path,
                                                            config=electra_config,
                                                            num_labels=359)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()


while 1:
  sent = input('\nQuestion: ') # '요즘 기분이 우울한 느낌이에요'
  data = koelectra_input(tokenizer,sent, device,512)
  # print(data)

  output = model(**data)

  logit = output
  softmax_logit = nn.Softmax(logit).dim
  softmax_logit = softmax_logit[0].squeeze()

  max_index = torch.argmax(softmax_logit).item()
  max_index_value = softmax_logit[torch.argmax(softmax_logit)].item()


  #answer_list = answer[category[str(max_index)]]
  #answer_len= len(answer_list)-1
  #answer_index = random.randint(0,answer_len)
  #print(f'Answer: {answer_list[answer_index]}, index: {max_index}, value: {max_index_value}')
  print(f'index: {category[max_index]}, value: {max_index_value}')
  print('-'*50)
# print('argmin:',softmax_logit[torch.argmin(softmax_logit)])

Some weights of the model checkpoint at monologg/koelectra-base-discriminator were not used when initializing koElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing koElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing koElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of koElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized


Question: 롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.
index: 감정/감정조절이상, value: 11.001134872436523
--------------------------------------------------

Question: 순간순간 감정조절을 못해요.
index: 감정/감정조절이상, value: 10.957347869873047
--------------------------------------------------

Question: 내 마음 이해하겠어?
index: 일반대화, value: 11.971019744873047
--------------------------------------------------

Question: 특별히 무슨 일이 있거나 그런 건 아니에요.
index: 원인/없음, value: 7.531722545623779
--------------------------------------------------

Question: 바쁠 땐 아무 생각이 없는데 한가해질 때면 우울함이 밀려와요.
index: 감정/우울감, value: 12.761611938476562
--------------------------------------------------

Question: 자꾸 걔 눈치를 보게 되고, 말하다가 내가 실수할까봐 엄청 신경쓰게 되고…
index: 감정/신경쓰임, value: 10.4808931350708
--------------------------------------------------

Question: 자꾸만 뭔가를 던지려고 해, 실수하면 나를 죽일거야.
index: 증상/피해망상, value: 12.111274719238281
--------------------------------------------------

Question: 이제는 우리가 헤어져야할 시간, 다음에 또 만나요
index: 배경/연애, value: 7.636196613311768
------------

KeyboardInterrupt: ignored